# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Импортируем необходимые для выполнения проекта библиотеки и функции:

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Загрузим датасеты:

In [2]:
df1 = pd.read_csv('/datasets/geo_data_0.csv')
df2 = pd.read_csv('/datasets/geo_data_1.csv')
df3 = pd.read_csv('/datasets/geo_data_2.csv')

Теперь последовательно изучим каждый из них. Создадим функцию, которая выдаст нам необходимую информацию о каждом датасете:

In [3]:
def explanations(df):
    display(df.head())
    df.info()
    print('Всего пропусков:', df.isna().sum().sum())
    print('Всего дубликатов:', df.duplicated().sum())
    print('Максимальные значения признаков:')
    print(df[["f0","f1","f2"]].max())
    print('Минимальные значения признаков:')
    print(df[["f0","f1","f2"]].min())

In [4]:
explanations(df1)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
Всего пропусков: 0
Всего дубликатов: 0
Максимальные значения признаков:
f0     2.362331
f1     1.343769
f2    16.003790
dtype: float64
Минимальные значения признаков:
f0    -1.408605
f1    -0.848218
f2   -12.088328
dtype: float64


In [5]:
explanations(df2)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
Всего пропусков: 0
Всего дубликатов: 0
Максимальные значения признаков:
f0    29.421755
f1    18.734063
f2     5.019721
dtype: float64
Минимальные значения признаков:
f0   -31.609576
f1   -26.358598
f2    -0.018144
dtype: float64


In [6]:
explanations(df3)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
Всего пропусков: 0
Всего дубликатов: 0
Максимальные значения признаков:
f0     7.238262
f1     7.844801
f2    16.739402
dtype: float64
Минимальные значения признаков:
f0    -8.760004
f1    -7.084020
f2   -11.970335
dtype: float64


**ВЫВОД**

В процессе подготовки данных были загружены 3 датасета с данными геологоразведки трёх регионов. Пропусков и дубликатов нет. Типы данных в каждом столбце - корректны.

Значения и их разбросы в столбцах признаков отличаются - притом для каждого датасета они тоже разные. Чтобы алгоритм считал все признаки одинаково важными, необходимо будет произвести масштабирование для каждого датасета. Объединять их не будем, а продолжим работу с тремя отдельными таблицами.

## Обучение и проверка модели

Для обучения и проверки модели необходимо произвести следующие операции:

* Разбить данные на обучающую и валидационную выборку в соотношении 75:25.
* Обучить модель и сделать предсказания на валидационной выборке.
* Сохранить предсказания и правильные ответы на валидационной выборке.
* Напечатать на экране средний запас предсказанного сырья и RMSE модели.

Далее необходимо будет проанализировать результаты.
Зададим функцию:

In [7]:
def everything_you_want(df):
    features = df.drop(["product","id"], axis=1) #уберем целевой признак и id каждой скважины
    target = df["product"]
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    
    scaler = StandardScaler() #произведем масштабирование признаков
    features_train = scaler.fit_transform(features_train)
    features_valid = scaler.transform(features_valid)
    
    model_regression = LinearRegression()
    model_regression.fit(features_train, target_train)
    predictions_valid = model_regression.predict(features_valid)
    predictions_valid = pd.Series(predictions_valid)
    average_stock = sum(predictions_valid)/len(predictions_valid)
    rmse = (mean_squared_error(predictions_valid, target_valid))**(0.5)
    
    print(f'RMSE модели: {rmse:.4f}')
    print(f'Средний запас предсказанного сырья: {average_stock:.4f}')
    return (predictions_valid, target_valid.reset_index(drop=True), rmse, average_stock)

In [8]:
k=1
for i in [df1, df2, df3]:
    print(f'Регион {k}:')
    everything_you_want(i)
    k +=1
    print()

Регион 1:
RMSE модели: 37.5794
Средний запас предсказанного сырья: 92.5926

Регион 2:
RMSE модели: 0.8931
Средний запас предсказанного сырья: 68.7285

Регион 3:
RMSE модели: 40.0297
Средний запас предсказанного сырья: 94.9650



Теперь сохраним значения для каждого региона:

In [9]:
predictions_valid_1, target_valid_1, rmse_1, average_stock_1 = everything_you_want(df1)
predictions_valid_2, target_valid_2, rmse_2, average_stock_2 = everything_you_want(df2)
predictions_valid_3, target_valid_3, rmse_3, average_stock_3 = everything_you_want(df3)

RMSE модели: 37.5794
Средний запас предсказанного сырья: 92.5926
RMSE модели: 0.8931
Средний запас предсказанного сырья: 68.7285
RMSE модели: 40.0297
Средний запас предсказанного сырья: 94.9650


**ВЫВОД**

Наименьший корень среднеквадратической ошибки у модели, предсказывающей поведение второго региона. Притом у данного региона наименьший средний запас предсказанного сырья. У первого и третьего регионов ситуация обратная: большой средний запас предсказываемого сырья, но и RMSE значительно больше. Нам необходимо рассчитать достаточный объем сырья для безубыточной работы, чтобы понять, какой из регионов подходит лучше.

## Подготовка к расчёту прибыли

Бюджет на разработку скважин в регионе равен 10 млрд рублей. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей. Необходимо определить, сколько тысяч баррелей нужно продать, чтобы разница между общим доходом и бюджетом была нулевой. Зная это, найдем точку безубыточности:

In [10]:
budget = 10**10
income_per_thousand = 450*10**3
n = np.ceil(budget/income_per_thousand) #округлим в сторону большего целого
print(f'Для безубыточной работы необходимо продать {n:.1f} тысяч баррелей')

Для безубыточной работы необходимо продать 22223.0 тысяч баррелей


При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки. Таким образом, необходимо, сколько в среднем тысяч баррелей должно быть на каждой точке, чтобы достичь точку безубыточности:

In [11]:
average_raw = np.ceil(n/200) #округлим в сторону большего целого
print(average_raw)

112.0


**ВЫВОД**

Средний запас предсказанного сырья для всех трех регионов меньше необходимого для безубыточной работы. Если мы просто поставим скважины - дело прогорит. Необходимо тщательно выбрать точки для скважин.

## Расчёт прибыли и рисков 

Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

In [12]:
def profit(predictions, target):
    best_predictions = predictions.sort_values(ascending=False) #разместили предсказания в порядке убывания
    best_target = target[best_predictions.index][:200] #выбрали целевые значения, соответствующие 200 лучших предсказаний
    revenue = best_target.sum() * income_per_thousand #суммарная выручка
    income = revenue - budget #прибыль
    return income

Перейдем к расчету рисков и прибыли каждого региона. Для определения распеделения прибыли, доверительного интервала и риска убытков применим технику Bootstrap с 1000 выборок:

In [13]:
def profit_and_risk(predictions, target):
    state = np.random.RandomState(12345)
    values = []
    for j in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        predictions_subsample = predictions[target_subsample.index]
        values.append(profit(predictions_subsample, target_subsample))
    
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean_rev = values.mean()
    risk = st.percentileofscore(values, 0)
        
    return ((int(lower), int(upper)), mean_rev, risk)

In [14]:
a = [predictions_valid_1, predictions_valid_2, predictions_valid_3]
b = [target_valid_1, target_valid_2, target_valid_3]
k = 1
for i,j in zip(a,b):
    interval, mean_revenue, risk = profit_and_risk(i, j)
    print(f'Регион {k}:')
    print(f'95%-й доверительный интервал: {interval}')
    print(f'Средняя прибыль: {mean_revenue:.0f}')
    print(f'Риск: {risk}')
    print()
    k +=1

Регион 1:
95%-й доверительный интервал: (-102090094, 947976353)
Средняя прибыль: 425938527
Риск: 6.0

Регион 2:
95%-й доверительный интервал: (68873225, 931547591)
Средняя прибыль: 515222773
Риск: 1.0

Регион 3:
95%-й доверительный интервал: (-128880547, 969706954)
Средняя прибыль: 435008363
Риск: 6.4



Среди регионов только во втором риски оказались ниже 2.5%, что удовлетворяет условию задачи. Средняя прибыль с одной точки составляет **515222773**. С учетом риска в 1% инвестиции окупятся сполна

# Вывод

В ходе выполнения работы были изучены данные пробы нефти в трех регионах. Пропуски и дубликаты отсутствовали.

Далее были составлены модели линейной регрессии для каждого региона с учетом дисбаланса классов. Найдены значения RMSE, среднего запаса предсказываемого сырья и точки безубыточности.

Следом с помощью техники Bootstrap для каждого были определены: средняя прибыль, 95%-й доверительный интервал и риск. Удовлетворяющим требования оказался регион №2. Таким образом, цитируя Макконахи в фильме "Золото" (2016), мы нашли золотую жилу!